## Visualiation

In [35]:
import requests

base_url = "http://127.0.0.1:8000"

# Create a new session
problem = {
    "problem_text": """In the triangle ABC, sides AB and CB have equal lengths and the measure of angle $\angle ABC$ is equal to 36 degrees. 
    What is the measure of angle $\angle BOC$ where O is the center of the circle goes through A, B, and C?"""
}

# Create a new session with the problem
response = requests.post(f"{base_url}/sessions", json=problem)
session_data = response.json()
print("Session ID:", session_data["session_id"])

# Get solution
response = requests.get(
    f"{base_url}/solution",
    json={"session_id": session_data["session_id"]}
)

if response.ok:
    solution = response.json()
    # print("Solution:", solution["solution_text"])
    # print("Moved to next question:", solution["moved_to_next"])
else:
    print("Error:", response.status_code, response.text)

Session ID: fde0026c-1f2d-4708-8a32-e550db4bd33c


In [ ]:
import google.generativeai as genai
GOOGLE_API_KEY=""
GEMINI_MODEL="gemini-2.0-flash"

genai.configure(api_key=GOOGLE_API_KEY)
generation_config = {
    "temperature": 0.0,
    "top_p": 0.5,
    "top_k": 40,
    "max_output_tokens": 2048,
}
model = genai.GenerativeModel(GEMINI_MODEL, generation_config=generation_config)

def clean_code(code_content):
    # Check if the first line starts with "```python", remove it if it does. Also, last line is "```"
    if code_content.startswith("```python"):
        code_content = code_content[10:]  # Remove the first line
    if code_content.endswith("```"):
        code_content = code_content[:-3]  # Remove the last line
    return code_content

#### 6. Viz by html

In [122]:
# self.prompt_plotly = """
# You are a master of geometry and data visualization. Given the following geometry problem and its solution, generate executable Plotly Python code that visualizes the final geometric construction described in the solution.
# Requirements:
# - Output must be valid Python code only, formatted for direct use in a Jupyter notebook.
# - Do not include any comments, markdown, explanations, or extra text.
# - The visualization should clearly represent the key geometric elements from the solution.

# Problem: {init_problem}
# Solution: {tutor_solution}
# """

# self.prompt_jsxgraph = """
# You are an expert in geometry and interactive visualization. Given the following geometry problem and its solution, generate JSXGraph JavaScript code to visually illustrate the final construction described in the solution.
# Requirements:
# - Output must be valid JSXGraph code only, ready to run in a browser environment or inside a JS-capable notebook.
# - Do not include any comments, explanations, or extra text.
# - Use JSXGraph elements (points, lines, circles, polygons, etc.) to accurately reflect the solution.

# Problem: {init_problem}
# Solution: {tutor_solution}
# """

# self.prompt_draw_steps = """
# You are an expert in geometry teacher. Given the following geometry problem and its solution,
# create a series of steps to draw the geometric construction described in the solution.
# The output should be a list of steps, each describing a specific action to take in order to recreate the construction.
# Do not include any code or technical details, just the steps in plain language.
# Always link to the coordinates of the points in the solution to support the drawing steps.

# Problem: {init_problem}
# Solution: {tutor_solution}
# """

# self.prompt_drawing_exec = """
# You are a master of geometry and data visualization using {language}.
# Following the steps below, generate executable code that visualizes the final geometric construction described in the solution.
# The output must be valid code that can be run in a Jupyter notebook or a web environment.
# Do not include any comments, markdown, explanations, or extra text.
# Steps: {drawing_steps}
# Visualization code:
# """

In [129]:
# - Add all the annotations and labels to highlight the key points and constructions.
class Prompt:
    def __init__(self):
        self.gen_asymptote = """
        You are a master of Asymptote: The Vector Graphics Language, who creates visualizations to solve the given geometry problem. 
        The below is a geometry problem, and its solution. Use your expertise to generate an Asymptote code that visualizes the solution.
        Requirements:
        - The output should be a valid Asymptote code that can be rendered to visualize the problem and solution.
        - Do not include any redundant text, just the Asymptote code.
        - Use the `geometry_reasoning` to identify the key points, lines, and other geometric elements that need to be visualized.
        - Utilize the `drawing_steps` to accurately depict the geometric construction.
        - If the problem can be drawn by Asymptote functions without actual coordinates, just use the Asymptote functions to draw the geometric construction.
        - Start the answer with: import olympiad; import settings; size(640);

        Key Points: {key_points}
        geometry_reasoning: {geometry_reasoning}
        drawing_steps: {drawing_steps}
        code:
        """

        self.prompt_code_revision = """
        You are an expert in code revision and optimization. Given the following {language} code and its errors.
        Rewrite the code to fix the errors and improve its quality.

        Requirements:
        - Reason step by step to ensure the code is correct and efficient.
        - Output must be the revised and valid code only, formatted for direct use.
        - Do not include any comments, explanations, or extra text.

        Current code: {code_content}
        Current code errors: {code_errors}
        Revised code:
        """
        
        self.get_key_points = """
        You are an expert in geometry problem solving. Given the following geometry problem and its solution,
        identify and extract the key points, objects that are essential for understanding and solving the problem.
        Requirements:
        - The output should be a list of key points, each described in a clear and concise manner.
        - Always link to the coordinates of the points in the solution to support the key points.
        - Just focus on the key points and return only key points.
        - No markdown marks.
        
        Example:
        Problem: A triangle ABC with sides AB and CB equal, angle ABC = 60 degrees.
        Solution: The measure of angle BAC is 60 degrees.
        List of key points:
        - Point A, B, C are vertices of the triangle ABC: These points define the triangle ABC.
        - AB = CB: This equality indicates that triangle ABC is an isosceles triangle with base AC
        - Angle ABC = 60 degrees: This is the vertex angle of the isosceles triangle. This is given in the problem statement.
        - Angle BAC = 60 degrees: This is the angle BAC, which is equal to angle ABC in an isosceles triangle.

        New problem format:
        Problem: {init_problem}
        Solution: {tutor_solution}
        List of key points:
        """

        self.set_coordinations = """
        You are an expert in geometry problem solving. Given the following geometry problem and its solution,
        set the coordinates of the points in the solution to support the key points. If the problem and the solution do not have coordinates, 
        generate appropriate coordinates for the problem.
        Requirements:
        - The output should be a list of coordinates, each described in a clear and concise manner
        - These coordinates should be linked to the key points in the solution.
        - Just focus on the coordinates and return only coordinates.
        - No markdown marks.
        - The coordinates should be in the format of (x, y) and must be carefully calculated to ensure they are correct.

        Problem: {init_problem}
        Solution: {tutor_solution}
        List of key points: {key_points}
        Coordinates of the points in the solution:
        """

        self.geometry_reasoning = """
        You are an expert in geometry problem solving. Given the following geometry problem and its solution.
        From a mathematical perspective, answer what are the objects whose coordinates can be randomly generated in the solution.
        And what are the objects that depend on the above objects when drawing the geometric construction.
        With the random objects, always generate coordinates for the points in the solution.
        With the dependency objects, dont give any coordinates, just describe them and its relationship with the random objects.
        Requirements:
        - The output should be a list of objects, each described in a clear and concise manner.
        - Always link to the coordinates of the points in the solution to support the objects.
        - Just focus on the objects and return only objects.
        - No markdown marks or redundant text.

        Problem: {init_problem}
        Solution: {tutor_solution}
        Key Points: {key_points}
        Reference Drawing Steps: {drawing_steps}
        """

        self.create_drawing_steps = """
        You are an expert in geometry problem solving. Given the following geometry's solution, key points,
        create a series of steps to draw the geometric construction described in the solution.
        Requirements:
        - The output should be a list of steps, each describing a specific action to take in order to recreate the construction.
        - Each step should be clear and concise, focusing on the geometric construction.
        - Always link to the coordinates of the points in the solution to support the drawing steps.
        - No markdown marks.
        - Describe in computer language instead of human language, showing the programming logically.

        Problem: {init_problem}
        Solution: {tutor_solution}
        Key Points: {key_points}
        Drawing steps:
        """

        self.gen_html_visualization = """
        You are a master of HTML-based geometry visualizations, using technologies like SVG, HTML5 Canvas, or D3.js to create clear, interactive, and accurate illustrations.
        The input below includes a geometry problem, its solution, and key steps in reasoning and drawing. Use your expertise to generate an HTML-based visualization (with embedded JavaScript or SVG as needed) that illustrates the solution.

        Requirements:
        - Output a valid HTML code block that renders a geometric diagram visualizing the problem and its solution.
        - Do not include any explanatory text outside the HTML code — only the HTML content itself.
        - Include all necessary tags (like <svg>, <canvas>, or <script>) so the code is standalone and renderable in a browser.
        - Use the geometry reasoning and drawing steps to identify and visualize key points, lines, circles, and other geometric features.
        - Label all important points clearly.
        - Make sure the diagram is centered and scaled appropriately for clarity.
        - Prefer simple and clean style: black lines, labeled points, optional color highlights (e.g., blue for constructions, red for key features).

        Tips:
        - Use <circle>, <line>, <text> inside <svg> for basic drawings.
        - Use coordinates based on the geometry reasoning and drawing steps.
        - Ensure all key points and constructions from the solution are visible and logical.

        Problem: {init_problem}
        Solution: {tutor_solution}
        Key Points: {key_points}
        Geometry Reasoning: {geometry_reasoning}
        Drawing Steps: {drawing_steps}
        HTML code:
        """


        self.gen_python_code = """
        You are a master of geometry and data visualization. Given the following geometry problem and its solution, 
        generate executable Plotly Python code that visualizes the final geometric construction described in the solution.
        Requirements:
        - Output must be valid Python code only, formatted for direct use in a Jupyter notebook.
        - The visualization should clearly represent the key geometric elements from the solution.
        - Follow the steps
        - Refer to the list of key points and coordinates to accurately depict the drawing steps.

        Problem: {init_problem}
        Solution: {tutor_solution}
        Key Points: {key_points}
        Coordinates: {coordinates}
        Drawing Steps: {drawing_steps}
        Visualization code:
        """
    
        self.save_to_svg = """
        You are an expert in visualization, the following is a Python code that generates a visualization of a geometry problem.
        Add lines of code to save the visualization to an SVG file. The path to save the SVG file is {file_path}, turn off fig.show()
        Current code: {code_content}
        Revised code:
        """

# Prompt to computer language instead of human language
# "programming logically"

In [130]:
class VizSolver:
    def __init__(self, session_id, init_problem, tutor_solution, code_lang="python"):
        self.session_id = session_id
        self.init_problem = init_problem
        self.tutor_solution = tutor_solution
        
        self.keypoints = None # Done
        self.annotations = None
        self.coordinates = None # Done
        self.drawing_guides = None
        self.lang = code_lang
        self.code = ""
        self.code_err = None
        self.code_exec = None
        self.revised_code = ""
        self.asymptote_code = ""

    def get_key_points(self):
        self.keypoints = model.generate_content(Prompt().get_key_points.format(init_problem=self.init_problem, tutor_solution=self.tutor_solution)).text

    def set_coordinates(self):
        self.coordinates = model.generate_content(Prompt().set_coordinations.format(init_problem=self.init_problem, 
                                                                                  tutor_solution=self.tutor_solution, 
                                                                                  key_points=self.keypoints)).text
    
    def create_drawing_steps(self):
        self.drawing_guides = model.generate_content(Prompt().create_drawing_steps.format(init_problem=self.init_problem, 
                                                                                        tutor_solution=self.tutor_solution,
                                                                                        key_points=self.keypoints)).text
    
    def get_geometry_reasoning(self):
        self.geometry_reasoning = model.generate_content(Prompt().geometry_reasoning.format(init_problem=self.init_problem, 
                                                                                           tutor_solution=self.tutor_solution, 
                                                                                           key_points=self.keypoints, 
                                                                                           drawing_steps=self.drawing_guides)).text
        
    def gen_code(self):
        self.code = model.generate_content(Prompt().gen_python_code.format(init_problem=self.init_problem,
                                                                            tutor_solution=self.tutor_solution,
                                                                            key_points=self.keypoints,
                                                                            coordinates=self.coordinates,
                                                                            drawing_steps=self.drawing_guides)).text
        self.code = clean_code(self.code)
        try:
            exec(self.code)
            self.code_exec = True
        except Exception as e:
            self.code_err = str(e)
            self.code_exec = False
        self.revised_code = model.generate_content(Prompt().prompt_code_revision.format(language=self.lang, 
                                                                                        code_content=self.code, 
                                                                                        code_errors=self.code_err)).text
        self.revised_code = clean_code(self.revised_code)

    def gen_asymptote_code(self):
        prompt = Prompt().gen_asymptote.format(
                                                                                # init_problem=self.init_problem, 
                                                                                #   tutor_solution=self.tutor_solution, 
                                                                                  key_points=self.keypoints, 
                                                                                  geometry_reasoning=self.geometry_reasoning, 
                                                                                  drawing_steps=self.drawing_guides)
        print("Asymptote prompt:", prompt)
        self.asymptote_code = model.generate_content(Prompt().gen_asymptote.format(
                                                                                # init_problem=self.init_problem, 
                                                                                #   tutor_solution=self.tutor_solution, 
                                                                                  key_points=self.keypoints, 
                                                                                  geometry_reasoning=self.geometry_reasoning, 
                                                                                  drawing_steps=self.drawing_guides)).text
        self.asymptote_code = clean_code(self.asymptote_code)

    def gen_html_visualization(self):
        self.html_code = model.generate_content(Prompt().gen_html_visualization.format(init_problem=self.init_problem, 
                                                                                        tutor_solution=self.tutor_solution, 
                                                                                        key_points=self.keypoints, 
                                                                                        geometry_reasoning=self.geometry_reasoning, 
                                                                                        drawing_steps=self.drawing_guides)).text
        
    def save_to_svg(self, file_path="drawing.svg"):
        self.save_svg_code = model.generate_content(Prompt().save_to_svg.format(
            file_path=file_path,
            code_content=self.revised_code,
        )).text
        self.save_svg_code = clean_code(self.save_svg_code)
        try:
            exec(self.save_svg_code)
            return True
        except Exception as e:
            print("Error saving SVG:", e)
            return False
        
    def viz_reasoning(self):
        self.get_key_points()
        self.create_drawing_steps()
        self.get_geometry_reasoning()

In [131]:
VizS = VizSolver(
    session_id=session_data["session_id"],
    init_problem=problem["problem_text"],
    tutor_solution=solution["solution_text"],
    code_lang="python"
)

VizS.viz_reasoning()
print("Key Points:", VizS.keypoints)
print("="* 50)
print("Drawing Steps:", VizS.drawing_guides)
print("="* 50)
print("Geometry Reasoning:", VizS.geometry_reasoning)

Key Points: - Point A, B, C are vertices of the triangle ABC: These points define the triangle ABC.
- AB = CB: This equality indicates that triangle ABC is an isosceles triangle with base AC.
- Angle ABC = 36 degrees: This is the vertex angle of the isosceles triangle ABC.
- Angle BAC = Angle BCA = 72 degrees: These are the base angles of the isosceles triangle ABC, calculated as (180 - 36) / 2.
- Point O is the center of the circle passing through A, B, and C: This point is the center of the circumcircle of triangle ABC.
- Angle BOC is the central angle subtended by arc BC: This angle is formed by the radii OB and OC.
- Angle BOC = 2 * Angle BAC = 144 degrees: This is based on the central angle theorem, where the central angle is twice the inscribed angle subtending the same arc.

Drawing Steps: 1.  Define point B at coordinate (0,0).
2.  Define point A such that AB = CB and angle ABC = 36 degrees. Let A be at coordinate (x,y).
3.  Define point C such that CB = AB and angle ABC = 36 d

In [ ]:
VizS.gen_html_visualization()

In [49]:
# VizS.html_code to html file
with open("visualization.html", "w") as f:
    f.write(VizS.html_code)

### Plotly Python Visualization

In [ ]:
VizS.gen_code()

In [ ]:
# Save the code to an SVG file
if VizS.save_to_svg(file_path="drawing.svg"):
    print("SVG saved successfully.")
else:
    print("Failed to save SVG.")

Error saving SVG: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido

Failed to save SVG.


### Asymptote Visualization

In [132]:
VizS.gen_asymptote_code()
print("Asymptote Code:", VizS.asymptote_code)

Asymptote prompt: 
        You are a master of Asymptote: The Vector Graphics Language, who creates visualizations to solve the given geometry problem. 
        The below is a geometry problem, and its solution. Use your expertise to generate an Asymptote code that visualizes the solution.
        Requirements:
        - The output should be a valid Asymptote code that can be rendered to visualize the problem and solution.
        - Do not include any redundant text, just the Asymptote code.
        - Use the `geometry_reasoning` to identify the key points, lines, and other geometric elements that need to be visualized.
        - Utilize the `drawing_steps` to accurately depict the geometric construction.
        - If the problem can be drawn by Asymptote functions without actual coordinates, just use the Asymptote functions to draw the geometric construction.
        - Start the answer with: import olympiad; import settings; size(640);

        Key Points: - Point A, B, C are vertices

## Asymptote Visualization

In [ ]:
import requests
from html.parser import HTMLParser

class AsyLinkParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.asy_files = []

    def handle_starttag(self, tag, attrs):
        if tag == "a":
            for attr in attrs:
                if attr[0] == "href" and attr[1].endswith(".asy"):
                    self.asy_files.append(attr[1])

gallery_url_base = "http://asymptote.sourceforge.net/gallery/"
parser = AsyLinkParser()
response = requests.get(gallery_url_base + ".index1.html")
parser.feed(response.text)
parser.asy_files[:10]

['CAD1.asy',
 'CDlabel.asy',
 'Hobbycontrol.asy',
 'Hobbydir.asy',
 'Pythagoras.asy',
 'advection.asy',
 'alignbox.asy',
 'arclength.asy',
 'axialshade.asy',
 'basealign.asy']

In [13]:
asy_data = []
skip_urls = ["intro.asy",     # This file is missing
             "exp3.asy",      # Compiling leads to a stack overflow?
             "galleon.asy",   # Requires external data
             "westnile.asy",  # Requires external data
             ]
# These keywords suggest additional asymptote modules or tampering
# with settings in an unexpected way.
skip_list = ["cpkcolors", "embed", "logo", "settings"]
for gallery_url in parser.asy_files[:10]:
    if gallery_url in skip_urls:
        continue
    asy_code = requests.get(gallery_url_base + gallery_url).text
    if not any([skip_word in asy_code for skip_word in skip_list]):
        asy_data.append({"code": asy_code, "name": gallery_url})

In [15]:
print(len(asy_data))
asy_data[0]

8


{'code': 'import CAD;\n\nsCAD cad=sCAD.Create();\n\n// Freehand line\ndraw(g=cad.MakeFreehand(pFrom=(3,-1)*cm,(6,-1)*cm),\n     p=cad.pFreehand);\n\n// Standard measurement lines\ndraw(g=box((0,0)*cm,(1,1)*cm),p=cad.pVisibleEdge);\ncad.MeasureParallel(L="$\\sqrt{2}$",\n                    pFrom=(0,1)*cm,\n                    pTo=(1,0)*cm,\n                    dblDistance=-15mm);\n\n// Label inside,shifted to the right; arrows outside\ndraw(g=box((2,0)*cm,(3,1)*cm),p=cad.pVisibleEdge);\ncad.MeasureParallel(L="1",\n                    pFrom=(2,1)*cm,\n                    pTo=(3,1)*cm,\n                    dblDistance=5mm,\n                    dblLeft=5mm,\n                    dblRelPosition=0.75);\n\n// Label and arrows outside\ndraw(g=box((5,0)*cm,(5.5,1)*cm),p=cad.pVisibleEdge);\ncad.MeasureParallel(L="0.5",\n                    pFrom=(5,1)*cm,\n                    pTo=(5.5,1)*cm,\n                    dblDistance=5mm,\n                    dblLeft=10mm,\n                    dblRelPositi

In [ ]:
from IPython.display import display, HTML
import asymagic

def display_asy(asy_data, start_index=0, end_index=-1):
    """Display subset of asymptote examples"""
    asy_codes = [tup["code"] for tup in asy_data[start_index : end_index]]
    print(asy_codes[0][:10])
    with asymagic.TemporaryAsymptoteFile(asy_codes) as tmp_asy:
        for i, asy_file in enumerate(tmp_asy.asy_files):
            display(HTML("<a href='{base}/{asy}'>{asy}</a>".format(
                base=gallery_url_base, asy=asy_data[i + start_index]["name"])))
            display(asymagic.run_asy_file(asy_file[0]))
display_asy(asy_data, 3, 5)


In [50]:
import subprocess
import os

def compile_asymptote_to_png(asy_code, output_filename="output.png", output_dir=".", size="6cm", render_quality=4):
    """
    Compiles Asymptote code into a PNG image.

    Args:
        asy_code (str): The Asymptote code as a string.
        output_filename (str): The desired name for the output PNG file.
        output_dir (str): The directory where the .asy and .png files will be saved.
        size (str): Asymptote 'size' parameter (e.g., "6cm", "200").
        render_quality (int): Asymptote '-render' option for anti-aliasing quality.
                              Higher values mean better quality but slower compilation.
    Returns:
        bool: True if compilation was successful, False otherwise.
        str: Path to the generated PNG file if successful, or an error message.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    asy_filepath = os.path.join(output_dir, "temp_asymptote_code.asy")
    png_filepath = os.path.join(output_dir, output_filename)

    try:
        # 1. Write the Asymptote code to a .asy file
        with open(asy_filepath, "w") as f:
            f.write(asy_code)

        # 2. Construct the Asymptote command
        # -f png: output format is PNG
        # -render X: anti-aliasing quality (e.g., 4 for good quality)
        # -V: suppress viewing the output after compilation (useful for automation)
        # -noV: same as -V
        # -prc: suppress creating a PRC file (if 3D)
        # -noprc: same as -prc
        command = [
            "asy",
            "-f", "png",
            "-render", str(render_quality),
            "-V", # Suppress viewing
            "-noprc", # Suppress PRC file generation
            asy_filepath,
            f'-o{png_filepath}', # Specify output file directly
            f'-c",size({size});"' # Add size command. Note the comma and quotes for multi-option -c
        ]

        # 3. Execute the Asymptote command
        result = subprocess.run(
            command,
            capture_output=True,
            text=True,
            check=False  # Don't raise an exception for non-zero exit codes immediately
        )

        # 4. Check the result
        if result.returncode == 0:
            print(f"Asymptote compilation successful: {png_filepath}")
            return True, png_filepath
        else:
            print(f"Asymptote compilation failed with error code {result.returncode}")
            print("Stdout:", result.stdout)
            print("Stderr:", result.stderr)
            return False, result.stderr

    except FileNotFoundError:
        return False, "Error: 'asy' command not found. Is Asymptote installed and in your system PATH?"
    except Exception as e:
        return False, f"An unexpected error occurred: {e}"
    finally:
        # Clean up the temporary .asy file (optional, but good practice)
        if os.path.exists(asy_filepath):
            os.remove(asy_filepath)

asy_code_example = """
import graph;
size(6cm);
pair A = (0,0);
pair B = (2,1);
pair C = (1,3);
draw(A--B--C--cycle, blue+1pt);
label("$A$", A, SW);
label("$B$", B, SE);
label("$C$", C, N);
dot(A); dot(B); dot(C);
"""

success, result_path = compile_asymptote_to_png(
    asy_code_example,
    output_filename="my_asymptote_plot.png",
    output_dir="asy_output",
    size="8cm", # Override default size
    render_quality=8
)

if success:
    print(f"PNG file created at: {result_path}")
else:
    print(f"Error: {result_path}")

Asymptote compilation successful: asy_output/my_asymptote_plot.png
PNG file created at: asy_output/my_asymptote_plot.png
